In [1]:
# imports
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd #https://www.dataquest.io/blog/large_files/pandas-cheat-sheet.pdf
import seaborn as sns
import itertools

In [2]:
# read input
import csv

filename_train = "data/training_set_VU_DM_2014.csv"
# filename_test = "data/training_set_VU_DM_2014.csv"

df_train = pd.read_csv(filename_train)
# df_test = pd.read_csv(filename_test)

In [3]:
# create smaller sample set to make for feasible testing
df_sample = df_train.sample(frac=0.05)

In [4]:
# helper functions
def make_daypart(hours):
    if hours >= 8 and hours < 12:
        return 0 #"morning"
    elif hours >= 12 and hours < 18:
        return 1 #"day"
    elif hours >= 18 and hours < 24:
        return 2 #"evening"
    else:
        return 3 #"night"

def make_distance(km):
    if km >= 0 and km < 200:
        return 0 #"close"
    elif km >= 200 and km < 1000:
        return 1 #"medium"
    elif km >= 1000:
        return 2 #"far"
    else:
        return -1 #"unknown"

def make_price(price):
    if price >= 0 and price < 50:
        return 0 #"cheap"
    elif price >= 50 and price < 100:
        return 1 #"medium"
    elif price >= 100:
        return 2 #"expensive"
    else:
        return -1 #"unknown"

def make_length(length):
    if length >= 0 and length < 3:
        return 0 #"days"
    elif length >= 3 and length < 8:
        return 1 #"week"
    elif length >= 8 and length < 15:
        return 2 #"weeks"
    else:
        return 3 #"month"
    
def make_star_rating(rating):
    if math.isnan(rating):
        return -1
    else:
        return round(rating)
    
# return -1 if there is a competitor with cheaper price, 0 if same, 1 if all competitors are more expensive.
# only counts competitors that actually have room
# NB: SIGNIFICANTLY SLOWS DOWN THE DATA PREPPING
def make_comp_score(row):
    rv = 1
    for i in range(1,9):
        if float(row["comp%d_inv"%i]) == 0: #if competitor has room available
            rv = min(row["comp%d_rate"%i], rv)
    return rv
        

In [9]:
''' Prep a dataframe

    Args:
        df_in: Dataframe of the dataset
        in_place: Boolean, if True prepare the data in place, if False create a new dataframe. Default=True

    Returns:
        Dataframe with prepped data
'''
def prep_dataframe(df_in, in_place=True):
    df = df_in if in_place else df_in.copy()
    
    # convert date_time to datetime
    df["date_time"] = pd.to_datetime(df["date_time"])

    # create relevant date columns
    df["srch_day_part"] = (df["date_time"].dt.hour).apply(lambda row: make_daypart(row))
    df["srch_day"] = df["date_time"].dt.weekday
    df["srch_month"] = df["date_time"].dt.month
    df["srch_quarter"] = df["date_time"].dt.quarter
    df["srch_year"] = df["date_time"].dt.year
    
    # change the distance
    df["orig_destination_distance_categ"] = df["orig_destination_distance"].apply(lambda row: make_distance(row))

    # change the price
    df["price_usd_categ"] = df["price_usd"].apply(lambda row: make_price(row))

    # change children
    df["has_children"] = np.where(df['srch_children_count'] > 0, 1, 0)

    # change stay length
    df["srch_length_of_stay_categ"] = df["srch_length_of_stay"].apply(lambda row: make_length(row))
    
    # change starratings
    df["visitor_hist_starrating"] = df["visitor_hist_starrating"].apply(lambda row: make_star_rating(row))

    # change competitor score
    df["comp_all_rate"] = df[["comp%d_rate" % i for i in range(1,9)]].min(axis=1)
    df["comp_all_rate_avail"] = df.apply(make_comp_score, axis=1) #NB: Expensive calculation #TODO: optimize with df methods instead of this disgrace to humanity
    
    #TODO: Do something with the relationship between visitor_hist_adr_usd and price_usd
    #idea: check if same category (problem case: 0-200 is cheap, 201+ is expensive, what if prices are 199 and 202?)
    #idea: check if less than or more than
    #idea: check how many std's difference 
    
    # remove all NaN's from the dataframe
    df = df.fillna('') # untested, not sure if this makes errors later on, also takes very long
    
    return df

In [10]:
''' Get list of features which are to be EXCLUDED from the training data
    This is done the other way around so that if new things get added they are automatically included.

    Returns:
        List with features.
'''
def features_to_exclude():
    to_exclude = [
#         "srch_id",
        "date_time",
#         "site_id",
#         "visitor_location_country_id",
#         "visitor_hist_starrating",
        "visitor_hist_adr_usd",
#         "prop_country_id",
#         "prop_id",
#         "prop_starrating",
#         "prop_review_score",
#         "prop_brand_bool",
        "prop_location_score1",
        "prop_location_score2",
        "prop_log_historical_price",
        "position",
        "price_usd",
#         "promotion_flag",
#         "srch_destination_id",
        "srch_length_of_stay",
        "srch_booking_window",
        "srch_adults_count",
        "srch_children_count",
        "srch_room_count",
#         "srch_saturday_night_bool",
        "srch_query_affinity_score",
        "orig_destination_distance",
#         "random_bool",
        "comp1_rate",
        "comp1_inv",
        "comp1_rate_percent_diff",
        "comp2_rate",
        "comp2_inv",
        "comp2_rate_percent_diff",
        "comp3_rate",
        "comp3_inv",
        "comp3_rate_percent_diff",
        "comp4_rate",
        "comp4_inv",
        "comp4_rate_percent_diff",
        "comp5_rate",
        "comp5_inv",
        "comp5_rate_percent_diff",
        "comp6_rate",
        "comp6_inv",
        "comp6_rate_percent_diff",
        "comp7_rate",
        "comp7_inv",
        "comp7_rate_percent_diff",
        "comp8_rate",
        "comp8_inv",
        "comp8_rate_percent_diff",
        "click_bool",
        "gross_bookings_usd",
        "booking_bool",
#         "srch_day_part",
#         "srch_day",
#         "srch_month",
#         "srch_quarter",
#         "srch_year",
#         "orig_destination_distance_categ",
#         "price_usd_categ",
#         "has_children",
#         "srch_length_of_stay_categ",
#         "comp_all_rate",
#         "comp_all_rate_avail",
    ]
    return to_exclude

In [11]:
'''Create features and target

    Args:
        
    Returns:
        List of features
        List of targets 
'''
def create_features_and_target(df):
    features = df[[val for val in df.columns.values if val not in features_to_exclude()]].values
    target = df[["booking_bool", "click_bool"]].values
    return features, target

In [12]:
# Run the prep_dataframe
df_sample2 = prep_dataframe(df_sample, False)
df_sample2.head(20).transpose()

,1359639,4486764,1734357,239190,3901449,2586893,2748621,3513126,2355691,4631836,773543,1911942,3816899,1726377,1119373,39694,281971,3613086,3462645,2842393
srch_id,91054,301025,116457,16031,261871,173681,184526,235860,158165,310707,51739,128352,256272,115952,74717,2635,18895,242707,232534,190956
date_time,2012-11-14 02:31:17,2013-05-08 15:23:55,2013-04-28 09:11:01,2013-03-15 17:44:04,2013-03-07 22:13:51,2012-12-14 00:28:58,2012-11-28 07:54:23,2013-03-07 10:36:06,2013-02-16 18:16:20,2013-04-14 20:26:31,2013-05-30 10:45:17,2013-03-06 23:45:34,2012-12-03 11:51:49,2012-11-05 22:19:16,2013-06-03 18:29:41,2012-12-20 15:31:35,2013-04-30 12:05:28,2013-04-19 13:11:36,2013-03-23 15:34:00,2013-01-19 15:12:33
site_id,5,13,15,14,15,24,14,27,5,5,5,24,32,15,5,5,14,5,5,18
visitor_location_country_id,219,81,55,100,55,216,100,138,219,219,219,216,220,55,219,219,100,219,219,129
visitor_hist_starrating,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
visitor_hist_adr_usd,,,,,,,,,,,,,,,,,,,,
prop_country_id,219,98,219,219,181,26,219,138,219,219,219,219,221,181,219,219,219,219,219,215
prop_id,121744,84783,27525,67793,9216,40753,14912,119213,45525,98839,71987,37788,91355,28979,134405,26314,105408,13389,55303,127801
prop_starrating,4,2,4,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,2,2
prop_review_score,4,4,4.5,4.5,3,0,4,2,4.5,4.5,4.5,4.5,4,4.5,4.5,4.5,4,4.5,3,0


In [97]:
# create_features_and_target
features, target = create_features_and_target(df_sample2)

In [13]:
# This is what the classifier will be running on:
df_sample2[[col for col in df_sample2.columns.values if col not in features_to_exclude()]].head(20).transpose()

,1359639,4486764,1734357,239190,3901449,2586893,2748621,3513126,2355691,4631836,773543,1911942,3816899,1726377,1119373,39694,281971,3613086,3462645,2842393
srch_id,91054,301025,116457,16031,261871,173681,184526,235860,158165,310707,51739,128352,256272,115952,74717,2635,18895,242707,232534,190956
site_id,5,13,15,14,15,24,14,27,5,5,5,24,32,15,5,5,14,5,5,18
visitor_location_country_id,219,81,55,100,55,216,100,138,219,219,219,216,220,55,219,219,100,219,219,129
visitor_hist_starrating,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
prop_country_id,219,98,219,219,181,26,219,138,219,219,219,219,221,181,219,219,219,219,219,215
prop_id,121744,84783,27525,67793,9216,40753,14912,119213,45525,98839,71987,37788,91355,28979,134405,26314,105408,13389,55303,127801
prop_starrating,4,2,4,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,2,2
prop_review_score,4,4,4.5,4.5,3,0,4,2,4.5,4.5,4.5,4.5,4,4.5,4.5,4.5,4,4.5,3,0
prop_brand_bool,1,0,1,0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,0,0
promotion_flag,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
